In [ ]:
# xmlrpc_server.ipynb

import xmlrpc.server
import xmlrpc.client
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import datetime as dt
import pandas as pd
import pickle

def add_log(name):
    try:
        server_log.add_log(name)
    except ConnectionRefusedError:
        pass

server_log = xmlrpc.client.ServerProxy("http://localhost:8018")

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), requestHandler=RequestHandler)

# Тест
def ping():
    add_log("ping")
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    add_log("now")
    return dt.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    add_log("type")
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    add_log("sum")
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    add_log("pow")
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname, name, otchestvo, date):
    add_log("black_list_check")
    #Surname,Name,Patronym,Birth
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = frame.loc[((frame['Surname'] == sname) & (frame['Name' ] == name) & (frame['Patronym' ] == otchestvo)& (frame['Birth' ] == date))]
    if (exist.empty == False):
        return sname + " "+ name + " " + otchestvo + " " + date + ": " + "bad_boy"
    else:
        return sname + " "+ name + " " + otchestvo + " " + date + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Бинарная передача данных
def send_back_binary(bin_data):
    add_log("send_back_binary")
    data = bin_data.data
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) или монохромное изображение размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    add_log("color_inversion")
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    if len(img_arr.shape) == 3:
        for i in range(height):
            for j in range(width):
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
    else:
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = 255 - img_arr[i][j]
    
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

#отражение относительно вертикальной оси
def rotate(bin_data):
    add_log("rotate")
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    for i in range(height):
            img_arr[i] = img_arr[i][::-1]
    
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(rotate, 'rotate')

#бинаризация изображения
def binarization_image(img_arr, n):
    add_log("binarization_image")
    img_arr = pickle.loads(img_arr.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    if len(img_arr.shape) == 3:
        for i in range(height):
            for j in range(width):
                for k in range(3):
                    if img_arr[i][j][k] > n:
                        img_arr[i][j][k] = 255
                    else:
                        img_arr[i][j] = 0
    else:
        for i in range(height):
            for j in range(width):
                if img_arr[i][j] > n:
                    img_arr[i][j] = 255
                else:
                    img_arr[i][j] = 0
    
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(binarization_image, 'binarization_image')

def return_logs(operation, dt_left, dt_right):
    return server_log.return_logs(operation, dt_left, dt_right)
server.register_function(return_logs, 'return_logs')



print("Listening on port 8008...")
server.serve_forever()


Listening on port 8008...


127.0.0.1 - - [23/Oct/2023 12:55:04] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:55:16] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:55:28] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:55:41] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:55:53] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:56:05] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:56:17] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:56:29] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:56:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:56:54] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:57:06] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:58:02] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 12:59:43] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 13:03:45] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 13:06:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/202